In [1]:
#Download the deep sort algorithm and yolov3 weights

!git clone -q --recursive https://github.com/ZQPei/deep_sort_pytorch.git
!curl https://pjreddie.com/media/files/yolov3.weights -o yolov3.weights
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&id={1_qwTWdzT9dWNudpusgKavj_4elGgbkUN}" -o ckpt.t7


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  236M  100  236M    0     0  15.7M      0  0:00:15  0:00:15 --:--:-- 18.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    139      0 --:--:--  0:00:02 --:--:--   139
100 43.9M    0 43.9M    0     0  7768k      0 --:--:--  0:00:05 --:--:-- 24.4M


In [0]:
import os
import sys
from os.path import exists, join, basename

project = "deep_sort_pytorch"
sys.path.append(project)
sys.path.append(join(project, 'detector/YOLOv3'))

In [46]:
import cv2
import time

from detector.YOLOv3 import YOLOv3
from deep_sort import DeepSort
from utils.draw import draw_boxes

import IPython
from IPython.display import clear_output


yolo3 = YOLOv3("deep_sort_pytorch/detector/YOLOv3/cfg/yolo_v3.cfg","yolov3.weights","deep_sort_pytorch/detector/YOLOv3/cfg/coco.names", is_xywh=True)
deepsort = DeepSort("ckpt.t7")

Loading weights from yolov3.weights... Done!
Loading weights from ckpt.t7... Done!


In [0]:
#Method to play video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))
 


In [0]:
input_path = 'Lead-Assignment'
inputs = list()

In [0]:
for file in os.listdir(input_path):
    if file.endswith(".mp4"):
      inputs.append(file)

In [50]:
inputs

['combined_ch1_20190826123000_20190826143000.mp4-01.54.16.779-01.54.45.146.mp4',
 '1 Weird Things Caught on Security Cameras HD CCTV Footage In _Market 30s - 60s (G015z0RzhI8).mp4',
 'CNN has obtained videos from inside the Westgate Mall 42s - 53s (VOq6owl8k4M).mp4',
 'Bangalore murder near Phoenix mall_ daytime CCTV footage _ Phoenix mall Bangalore murder 0s - 25s (YGiOo9Y_sGs).mp4',
 'Man strolls through Brent shopping center carrying axe 0s - 50s (wfsdRXvA8Fw).mp4',
 'Rohtak_ CCTV footage shows scores of armed people marching 6s - 40s (CC_-VsFOX1o).mp4',
 'Black Cats_ CCTV camera Robbery in SBI Bank Chennai 7m7s - 9m55s (6pjn97i3TDQ).mp4',
 'combined_ch8_20190826123000_20190826143000.mp4-01.54.16.095-01.54.44.239.mp4',
 'combined_ch8_20190826123000_20190826143000.mp4-01.55.12.555-0.mp4',
 'Top 3 in CCTV Dance Challenge _ Month of August 9s - 1m50s (hk__8PwYo-U).mp4',
 'combined_ch8_20190826123000_20190826143000.mp4-01.55.12.555-01.55.37.656.mp4',
 'combined_ch1_20190826123000_201908

In [51]:
clear_output()
video = show_local_mp4_video(os.path.join(input_path,inputs[1]))
video

In [0]:
#Method to search the video for people and track them
def detect_from_vid(vid_file):
  video_capture = cv2.VideoCapture()
  if video_capture.open(vid_file):

    width, height = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    video_writer = cv2.VideoWriter(vid_file[:-15]+".avi", cv2.VideoWriter_fourcc(*'MJPG'), fps, (width, height))

    while video_capture.isOpened():
      ret, frame = video_capture.read()
      if not ret:
        break
        
      xmin, ymin, xmax, ymax = 0, 0, width, height
      im = frame[ymin:ymax, xmin:xmax, (2,1,0)]
      bbox_xywh, cls_conf, cls_ids = yolo3(im)

      if bbox_xywh is not None:
          mask = cls_ids==0
          bbox_xywh = bbox_xywh[mask]
          bbox_xywh[:,3] *= 1.2
          cls_conf = cls_conf[mask]
          outputs = deepsort.update(bbox_xywh, cls_conf, im)

          if len(outputs) > 0:
              bbox_xyxy = outputs[:,:4]
              identities = outputs[:,-1]
              frame = draw_boxes(frame, bbox_xyxy, identities, offset=(xmin,ymin))
              
      video_writer.write(frame)
    video_capture.release()
    video_writer.release()

  else:
    print("can't open the given input video file!")

In [0]:
for vid in inputs:
  detect_from_vid(os.path.join(input_path,vid))